In [1]:
import pennylane as qml
from pennylane import numpy as np

import nlopt

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
dev = qml.device("strawberryfields.fock", wires=4, cutoff_dim=4)

In [3]:
def layer(theta, phi, wires):
    M = len(wires)
    phi_nonlinear = np.pi / 2

    qml.Interferometer(
        theta, phi, np.zeros(M), wires=wires, mesh="triangular",
    )

    for i in wires:
        qml.Kerr(phi_nonlinear, wires=i)

In [4]:
@qml.qnode(dev)
def quantum_neural_net(var, x):
    wires = list(range(len(x)))

    # Encode input x into a sequence of quantum fock states
    for i in wires:
        qml.FockState(x[i], wires=i)

    # "layer" subcircuits
    for i, v in enumerate(var):
        layer(v[: len(v) // 2], v[len(v) // 2 :], wires)

    return [qml.expval(qml.NumberOperator(w)) for w in wires]

In [5]:
def square_loss(labels, predictions):
    term = 0
    for l, p in zip(labels, predictions):
        lnorm = l / np.linalg.norm(l)
        pnorm = p / np.linalg.norm(p)

        term = term + np.abs(np.dot(lnorm, pnorm.T)) ** 2

    return 1 - term / len(labels)

In [6]:
def cost(var, data_input, labels):
    predictions = np.array([quantum_neural_net(var, x) for x in data_input])
    sl = square_loss(labels, predictions)

    return sl

In [7]:
# Define the CNOT input-output states (dual-rail encoding) and initialize
# them as non-differentiable.

X = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 1, 0],
              [0, 1, 0, 1]], requires_grad=False)

Y = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 0, 1],
              [0, 1, 1, 0]], requires_grad=False)

In [8]:
num_layers = 2
M = len(X[0])
num_variables_per_layer = M * (M - 1)

rng = np.random.default_rng(seed=1234)
var_init = (4 * rng.random(size=(num_layers, num_variables_per_layer), requires_grad=True) - 2) * np.pi
print(var_init)

[[ 5.99038594 -1.50550479  5.31866903 -2.99466132 -2.27329341 -4.79920711
  -3.24506046 -2.2803699   5.83179179 -2.97006415 -0.74133893  1.38067731]
 [ 4.56939998  4.5711137   2.1976234   2.00904031  2.96261861 -3.48398028
  -4.12093786  4.65477183 -5.52746064  2.30830291  2.15184041  1.3950931 ]]


In [9]:
cost_grad = qml.grad(cost)

print_every = 1

# Wrap the cost so that NLopt can use it for gradient-based optimizations
evals = 0
def cost_wrapper(var, grad=[]):
    global evals
    evals += 1

    if grad.size > 0:
        # Get the gradient for `var` by first "unflattening" it
        var = var.reshape((num_layers, num_variables_per_layer))
        var = np.array(var, requires_grad=True)
        var_grad = cost_grad(var, X, Y)
        grad[:] = var_grad.flatten()
    cost_val = cost(var.reshape((num_layers, num_variables_per_layer)), X, Y)

    if evals % print_every == 0:
        print(f"Iter: {evals:4d}    Cost: {cost_val:.4e}")

    return float(cost_val)


# Choose an algorithm
opt_algorithm = nlopt.LD_LBFGS  # Gradient-based
# opt_algorithm = nlopt.LN_BOBYQA  # Gradient-free

opt = nlopt.opt(opt_algorithm, num_layers*num_variables_per_layer)

opt.set_min_objective(cost_wrapper)

opt.set_lower_bounds(-2*np.pi * np.ones(num_layers*num_variables_per_layer))
opt.set_upper_bounds(2*np.pi * np.ones(num_layers*num_variables_per_layer))

var = opt.optimize(var_init.flatten())
var = var.reshape(var_init.shape)

Iter:    1    Cost: 5.2344e-01
Iter:    2    Cost: 4.6269e-01
Iter:    3    Cost: 3.3963e-01
Iter:    4    Cost: 3.0214e-01
Iter:    5    Cost: 2.7352e-01
Iter:    6    Cost: 1.9481e-01
Iter:    7    Cost: 2.6425e-01
Iter:    8    Cost: 8.8005e-02
Iter:    9    Cost: 1.3520e-01
Iter:   10    Cost: 6.9529e-02
Iter:   11    Cost: 2.2332e-02
Iter:   12    Cost: 5.4051e-03
Iter:   13    Cost: 1.7288e-03
Iter:   14    Cost: 5.7472e-04
Iter:   15    Cost: 2.1946e-04
Iter:   16    Cost: 8.5438e-05
Iter:   17    Cost: 3.9276e-05
Iter:   18    Cost: 1.8697e-05
Iter:   19    Cost: 8.7004e-06
Iter:   20    Cost: 3.7786e-06
Iter:   21    Cost: 1.5192e-06
Iter:   22    Cost: 7.0577e-07
Iter:   23    Cost: 3.1065e-07
Iter:   24    Cost: 1.4212e-07
Iter:   25    Cost: 6.3160e-08
Iter:   26    Cost: 2.5086e-08
Iter:   27    Cost: 1.2039e-08
Iter:   28    Cost: 4.6965e-09
Iter:   29    Cost: 1.6962e-09
Iter:   30    Cost: 6.1205e-10
Iter:   31    Cost: 2.4764e-10
Iter:   32    Cost: 1.2485e-10
Iter:   

In [10]:
print(f"The optimized parameters (layers, parameters):\n {var}\n")

Y_pred = np.array([quantum_neural_net(var, x) for x in X])
for i, x in enumerate(X):
    print(f"{x} --> {Y_pred[i].round(2)}, should be {Y[i]}")

The optimized parameters (layers, parameters):
 [[ 5.59646473 -0.76686268  6.28318531 -3.22867181 -1.61696116 -4.79794956
  -3.44889053 -2.68088815  5.65397191 -2.81207156 -0.59737993  1.39431044]
 [ 4.7105638   5.24800052  3.14152765  3.13959016  2.78451845 -3.92895253
  -4.38654718  4.65891554 -5.34964081  2.60705101  2.40425267  1.39415475]]

[1 0 1 0] --> [1. 0. 1. 0.], should be [1 0 1 0]
[1 0 0 1] --> [1. 0. 0. 1.], should be [1 0 0 1]
[0 1 1 0] --> [0. 1. 0. 1.], should be [0 1 0 1]
[0 1 0 1] --> [0. 1. 1. 0.], should be [0 1 1 0]


In [11]:
print(qml.draw(quantum_neural_net)(var_init, X[0]))


 0: ──|1⟩───────────────────────────────────────╭BS(5.32, 5.83)─────R(0)────────────────Kerr(1.57)─────────────────────────────────────────────────────────────────╭BS(2.2, -5.53)───R(0)─────────────Kerr(1.57)──────────────────────────────┤ ⟨n⟩ 
 1: ──|0⟩────────────────────╭BS(-1.51, -2.28)──╰BS(5.32, 5.83)────╭BS(-2.27, -0.741)───R(0)────────────Kerr(1.57)────────────────────────────────╭BS(4.57, 4.65)──╰BS(2.2, -5.53)──╭BS(2.96, 2.15)───R(0)────────────Kerr(1.57)──────────────┤ ⟨n⟩ 
 2: ──|1⟩──╭BS(5.99, -3.25)──╰BS(-1.51, -2.28)──╭BS(-2.99, -2.97)──╰BS(-2.27, -0.741)──╭BS(-4.8, 1.38)──R(0)────────Kerr(1.57)──╭BS(4.57, -4.12)──╰BS(4.57, 4.65)──╭BS(2.01, 2.31)──╰BS(2.96, 2.15)──╭BS(-3.48, 1.4)──R(0)────────Kerr(1.57)──┤ ⟨n⟩ 
 3: ──|0⟩──╰BS(5.99, -3.25)─────────────────────╰BS(-2.99, -2.97)──────────────────────╰BS(-4.8, 1.38)──R(0)────────Kerr(1.57)──╰BS(4.57, -4.12)───────────────────╰BS(2.01, 2.31)───────────────────╰BS(-3.48, 1.4)──R(0)────────Kerr(1.57)──┤ ⟨n⟩ 

